## mock_xray.ipynb

This file is used to generate X-ray mock images of TNG-Cluster primary zoom-in targets at snapshot 84.  
The codes are modified from example scripts provided in the TNG Lab.

Thanks to John ZuHone for preparing the original walkthrough for the LEM workshop in February 2022, and to Dylan Nelson for opening the TNG Lab resources to us.

**Input Requirements**

Local access to:
- Snapshot cutouts of TNG-Cluster primary zoom-in targets  
- CSV files listing TNG-Cluster primary zoom-in targets traced back to snapshot 84 from snapshot 89  

All source files are organized under the `data` folder.

In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress, pearsonr, spearmanr
import seaborn as sns
from scipy.optimize import curve_fit
import scipy.stats as stats
# Physics-related Packages
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
from astropy.constants import M_sun
import ast

# extra pack for X-ray mocking
from regions import RectangleSkyRegion
from astropy.coordinates import SkyCoord

from astropy import wcs
from astropy.io import fits
import yt
import pyxsim

In [ ]:
# get GroupPos and GroupR
# read CSV
cat_df = pd.read_csv('fof_halo_to_sub84.csv')

cat_df['GroupPos'] = cat_df['GroupPos'].apply(ast.literal_eval)

FOF_Halo_IDs = cat_df['FOF_Halo_ID_At84']
GroupPos = cat_df['GroupPos']
Group_R_Crit500 = cat_df['Group_R_Crit500']
SubhaloIDs = cat_df['Subhalo_ID_At84']

In [ ]:
from soxs.utils import soxs_cfg
soxs_cfg.set("soxs", "bkgnd_nH", "0.018") # avoid configparser error by specifying here
import soxs

In [ ]:
# get sim info
import requests
import time

headers = {"api-key": "API KEY"}


# get sim info
snapshot = 84
sim_url = "http://www.tng-project.org/api/TNG-Cluster/"
snap_url = f"http://www.tng-project.org/api/TNG-Cluster/snapshots/{snapshot}/"

r = requests.get(sim_url, headers=headers)
sim_info = r.json()

snap = requests.get(snap_url, headers=headers)
snap_info = snap.json()

redshift = snap_info['redshift']
factor_a = 1/(1+redshift)

In [ ]:
def generate_input_file(halo_index):
    # get halo info
    halo_id = int(FOF_Halo_IDs[halo_index])
    subhalo_id = int(SubhaloIDs[halo_index])
    #group_r = Group_R_Crit500[halo_index]

    halo = {
    "GroupPos": GroupPos[FOF_Halo_IDs==halo_id].iloc[0]
    }

    # generate header
    header = {
    "BoxSize": float(sim_info["boxsize"]),                 # [ckpc/h]
    "Time": factor_a,                                            # z = redshift 的 cosmic time
    "Redshift": redshift,
    "Omega0": float(sim_info["omega_0"]),                  # Omega matter
    "OmegaLambda": float(sim_info["omega_L"]),             # Omega Lambda
    "HubbleParam": float(sim_info["hubble"]),              # Hubble parameter
    }

    # 1 ckpc/h in cm (comoving kiloparsec per h) actually kpc in cm when simulation flag is turned on
    unit_length_cm = 1*(u.kpc).to(u.cm)
    
    # 1e10 Msun/h in grams actually 10^10M_sun in cm when simulation flag is turned on
    unit_mass_g = (1e10 * M_sun).to(u.g).value
    
    # 1 km/s to cm/s
    unit_velocity_cms = (1 * u.km / u.s).to(u.cm / u.s).value
    
    header_snap = {
        "UnitLength_in_cm": unit_length_cm,
        "UnitMass_in_g": unit_mass_g,
        "UnitVelocity_in_cm_per_s": unit_velocity_cms
    }
    
    # create mass table, only dm mass needed
    mass_table = np.zeros(6, dtype=np.float64)
    mass_table[1] = float(sim_info["mass_dm"])  # PartType1 = DM
    
    # add header snap
    header_snap["MassTable"] = mass_table

    # combine header and snap content
    filename = "halo_%d.hdf5" % halo_id
    #halo_file_name = f'/home/chuiyang/Illustris/TNGCluster_Cutout/snap84/cutout_sub{subhalo_id}_FOF{halo_id}.hdf5'
    halo_file_name = f'/users/ckong13/data/Chuiyang/TNGCluster_Cutout/snap{snapshot}/cutout_sub{subhalo_id}_FOF{halo_id}.hdf5'
    
    with h5py.File(halo_file_name) as cut_f:
        
        # print(cut_f['PartType0'].keys())
        num_gas = cut_f["PartType0/Coordinates"].shape[0]
        with h5py.File(filename,'w') as yt_f:
            for key in cut_f['PartType0'].keys():
                yt_f[f"PartType0/{key}"] = cut_f[f"PartType0/{key}"][:] 
                
            # some metadata that yt demands
            yt_f.create_group('Header')
            yt_f['Header'].attrs['NumFilesPerSnapshot'] = 1
            yt_f['Header'].attrs['MassTable'] = header_snap['MassTable']
            yt_f['Header'].attrs['BoxSize'] = header['BoxSize']
            yt_f['Header'].attrs['Time'] = header['Time']
            yt_f['Header'].attrs['Redshift'] = header['Redshift']
            yt_f['Header'].attrs['NumPart_ThisFile'] = np.array([num_gas,0,0,0,0,0])
            
            # Must have the next six for correct units
            yt_f["Header"].attrs["HubbleParam"] = header["HubbleParam"]
            yt_f["Header"].attrs["Omega0"] = header["Omega0"]
            yt_f["Header"].attrs["OmegaLambda"] = header["OmegaLambda"]
    
            # These correspond to the values from the TNG simulations
            yt_f["Header"].attrs["UnitLength_in_cm"] = header_snap['UnitLength_in_cm']
            yt_f["Header"].attrs["UnitMass_in_g"] = header_snap['UnitMass_in_g']
            yt_f["Header"].attrs["UnitVelocity_in_cm_per_s"] = header_snap['UnitVelocity_in_cm_per_s']

    return halo_id, halo_file_name, filename, halo

In [ ]:
# get hot gas
def hot_gas(pfilter, data):
    pfilter1 = data[pfilter.filtered_type, "temperature"] > 3.0e5
    pfilter2 = data["PartType0", "StarFormationRate"] == 0.0
    pfilter3 = data["PartType0", "GFM_CoolingRate"] < 0.0
    return (pfilter1 & pfilter2) & pfilter3

In [ ]:
def generate_source_model(emin=0.05, emax=4.0, nbins=8):
    
    source_model = pyxsim.CIESourceModel(
    "apec", emin, emax, nbins, ("hot_gas", "metallicity"),
    temperature_field=("hot_gas", "temperature"),
    emission_measure_field=("hot_gas", "emission_measure")
    )
    
    return source_model

In [ ]:
def get_width_mpc(halo_index):
    group_r = Group_R_Crit500[halo_index]
    group_r_q = group_r * factor_a * u.kpc / sim_info["hubble"]
    width_mpc = 2* group_r_q.to(u.Mpc)

    return float(width_mpc.value)

In [ ]:
logfile = "xray_chandra.log"
logging.basicConfig(
    level    = logging.INFO, 
    format   = "%(asctime)s [%(levelname)s] %(message)s",
    datefmt  = "%Y-%m-%d %H:%M:%S",
    handlers = [
        logging.FileHandler(logfile, mode="w"), 
        # logging.StreamHandler()               
    ]
)
logger = logging.getLogger()

logger.info("===== Start X-ray Mock=====")
start_all = time.time()

# define source parameter
exp_time = (2000., "ks") # exposure time
area = (600, "cm**2") # collecting area
redshift = snap_info['redshift']
# define instrument
instrument = "chandra_acisi_cy22"
ins_exp_time = (2000., "ks")

for halo_index in range(len(cat_df)):
# for halo_index in range(259,260):
    logger.info(f"--- start halo index {halo_index} ---")
    
    halo_id, halo_file_name, filename, halo = generate_input_file(halo_index)

    logger.info(f"  generate_input_file -> {filename}")
    
    ds = yt.load(filename)
    yt.add_particle_filter("hot_gas", function=hot_gas,
                           filtered_type='gas', requires=["temperature","density"])
    ds.add_particle_filter("hot_gas")
    
    # source model
    source_model = generate_source_model(emin=0.05, emax=4.0, nbins=8)
    
    # get width
    c = ds.arr([halo["GroupPos"][0], halo["GroupPos"][1], halo["GroupPos"][2]], "code_length")
    width = ds.quan(get_width_mpc(halo_index), "Mpc")
    le = c - 0.5*width
    re = c + 0.5*width
    #le = c - width
    #re = c + width
    box = ds.box(le, re)

    # generate photon and events
    n_photons, n_cells = pyxsim.make_photons(f"halo_{halo_id}_photons", box, redshift, area, exp_time, source_model)
    logger.info("make_photons: photons, cells")
    
    n_events = pyxsim.project_photons(f"halo_{halo_id}_photons", f"halo_{halo_id}_events", "x", (0.,0.),
                                      absorb_model="wabs", nH=0.01)
    logger.info("project_photons -> events")
    
    events = pyxsim.EventList(f"halo_{halo_id}_events.h5")
    events.write_to_simput(f"halo_{halo_id}", overwrite=True)

    # mock obs
    instrument = "chandra_acisi_cy22"
    soxs.instrument_simulator(f"halo_{halo_id}_simput.fits", f"halo_{halo_id}_evt.fits", ins_exp_time, instrument, (0.,0.), overwrite=True, foreground=False, ptsrc_bkgnd=False)

    # exposure calibration file
    exp_file  = f"halo_{halo_id}_expmap.fits"  
    soxs.make_exposure_map(f"halo_{halo_id}_evt.fits",
                       exp_file,
                       energy = 1.2,
                       overwrite=True)
    
    # plot
    soxs.write_image(f"halo_{halo_id}_evt.fits", f"halo_{halo_id}_img.fits", emin=0.1, emax=2.0, overwrite=True, expmap_file = exp_file)
    logger.info("finish write_image")    

    # 1) delete yt dataset
    ds = None

    # 2) delete biproductor
    try:
        del n_photons, events
    except NameError:
        pass

    # 3) delete local variables
    for var in ["box","c","width","le","re","source_model"]:
        if var in locals():
            del locals()[var]
    gc.collect()